In [19]:
from PIL import Image, ImageDraw
import dash
from dash import Dash, dcc, html, Input, Output, ctx
import io
import base64
import numpy as np
import pandas as pd
from dash import dash_table
from tensorflow.keras.models import load_model  

import torch

app = dash.Dash(__name__)

export_path = 'model_final_sept'
detect = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')  

class_names = ['Modern', 'Palaty', 'Brutalism', 'Constructivism', 'Socialist_classicism', 'Industrial_XIX', 'Panelka', 'XXI_century', 'Classicism', 'Church', 'Fortification']

app.layout = html.Div([
    html.Header(
            "This is app for recognition of architectural style of building by photo",
            style={"font-size": "30px", "textAlign": "center", 'font-family': 'Helvetica'},
        ),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select File (the image will be croped automaticaly for recognition)')
        ]),
        style={
            'width': '60%',
            'height': '80px',
            'lineHeight': '80px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px',
            'font-family': 'Helvetica'
        },
        # Allow multiple files to be uploaded
        multiple=False
    ),
    html.Div(id='output-image-upload'),
    html.Div(id='output-prediction-text'),
    html.Div(id='output-table-container')
])

def crop_building(image, bbox):
    x_min, y_min, x_max, y_max = bbox
    return image.crop((x_min, y_min, x_max, y_max))

def draw_boxes(image_path, boxes, class_names):
    #image = image_path.resize((640, 640))
    image = image_path
    draw = ImageDraw.Draw(image)

    for box in boxes[:1]:

        x, y, w, h, confidence, class_id = box
      #  x1, y1 = int(x - w/2), int(y - h/2)
      #  x2, y2 = int(x + w/2), int(y + h/2)
        x1, y1, x2, y2 = x, y, w, h

        # Draw rectangle
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        
        # Draw label
        label = f"{class_names[int(class_id)]}: {confidence:.2f}"
        draw.text((x1, y1 - 10), label, fill="red")
    
    return image

def resize_image_pil(img):
    # Convert the image string to PIL Image
    img = Image.open(io.BytesIO(base64.b64decode(img.split(",")[1])))  #.convert('L')
    img = img.convert("RGB")

    results = detect(img)
    check_detection = list(results.xyxy[0].numpy())


    if not check_detection:
        crop_fact = 'There is no building detected, probably style in table below. Returning uncropped image.'

        img = img.resize((640, 640))
        cropped_img = img
        imag_bbox = img

    else:
        crop_fact = 'Successful detection. Returning cropped image.'

        for bbox in results.xyxy[0].numpy():

            x_min, y_min, x_max, y_max, confidence, class_id = bbox

            x_min, y_min, x_max, y_max = map(int, [x_min, y_min, x_max, y_max])

            # Crop the building
            cropped_img = crop_building(img, (x_min, y_min, x_max, y_max))
            imag_bbox = draw_boxes(img, results.xyxy[0].numpy(), 'building')

    imag_bbox = imag_bbox.resize((640, 640))
  
    return cropped_img, imag_bbox, crop_fact

def process_and_predict(image_string):
    cropped_img, imag_bbox, crop_fact = resize_image_pil(image_string)
    
    # Load pre-trained model
    model = load_model(export_path)  
    
    img_resized = cropped_img.resize((256, 256))
    img_resized = img_resized.convert('L')
    img_resized = img_resized.convert("RGB")
    img_resized = np.array(img_resized)

    # Perform prediction using the model
    prediction = model.predict(np.expand_dims(img_resized / 255.0, axis=0))
    
    class_name = []
    probability = []

    for i in np.argsort(prediction, axis=1)[:, -3:]:
        [class_name.append(class_names[x]) for x in reversed(i)]
        [probability.append(str(np.round(prediction[0][x],2))) for x in reversed(i)]   

    class_idx = pd.DataFrame({'Style': class_name, 'Probability': probability})    

    return class_idx, cropped_img, imag_bbox, crop_fact  # Return the predicted class index

def pil_image_to_base64(img):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{img_str}"

@app.callback(Output('output-image-upload', 'children'),
              Output('output-table-container', 'children'),
              Output('output-prediction-text', 'children'),
              Input('upload-image', 'contents'), prevent_initial_call=True)

def update_output(contents):
    if contents is not None:
        # Process image and get prediction
        df, cropped_img, imag_bbox, crop_fact = process_and_predict(contents)
        
        # Convert the cropped image to base64
        cropped_img_base64 = pil_image_to_base64(imag_bbox)
        
        # Display cropped image
        children_img = html.Div([
            html.Img(src=cropped_img_base64, style={'width': '320px', 'height': '320px'}),
            html.Hr(),
            html.Div(id='output-prediction')
        ])
        
        # Create a table for the prediction results
        table = dash_table.DataTable(
            id='table',
            columns=[{'name': col, 'id': col} for col in df.columns],
            data=df.to_dict('records'),
                style_table={'textAlign': 'left', 'height': '150px', 'overflowY': 'auto', 'width': '350px', 'font-family': 'Helvetica'},
                style_cell={'textAlign': 'left', 'font-family': 'Helvetica'}
        )
        text_phrase = html.Div(crop_fact, style={'font-size': '20px', 'color': 'black', 'font-family': 'Helvetica'})
        # Return the updated image and prediction table
        return children_img, table, text_phrase
    
if __name__ == '__main__':
    app.run_server(debug=False, host='0.0.0.0', port=8080)

Using cache found in /Users/vladkozlovskiy/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-3 Python-3.11.7 torch-2.2.1 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


1/1 [==============================] - 0s 109ms/step
